In [2]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
import numpy as np
import pandas as pd

In [7]:
df = pd.read_csv("data/zomato_df_final_data.csv")

In [10]:
df.head()

,address,cost,cuisine,lat,link,lng,phone,rating_number,rating_text,subzone,title,type,votes,groupon,color,cost_2,cuisine_color
0,"371A Pitt Street, CBD, Sydney",50.0,"['Hot Pot', 'Korean BBQ', 'BBQ', 'Korean']",-33.876059,https://www.zomato.com/sydney/sydney-madang-cbd,151.207605,02 8318 0406,4.0,Very Good,CBD,Sydney Madang,['Casual Dining'],1311.0,False,#e15307,5.243902,#6f706b
1,"Shop 7A, 2 Huntley Street, Alexandria, Sydney",80.0,"['Cafe', 'Coffee and Tea', 'Salad', 'Poké']",-33.910999,https://www.zomato.com/sydney/the-grounds-of-a...,151.193793,02 9699 2225,4.6,Excellent,"The Grounds of Alexandria, Alexandria",The Grounds of Alexandria Cafe,['Café'],3236.0,False,#9c3203,7.560976,#6f706b
2,"Level G, The Darling at the Star, 80 Pyrmont ...",120.0,['Japanese'],-33.867971,https://www.zomato.com/sydney/sokyo-pyrmont,151.195210,1800 700 700,4.9,Excellent,"The Star, Pyrmont",Sokyo,['Fine Dining'],1227.0,False,#7f2704,10.650407,#6f706b
3,"Sydney Opera House, Bennelong Point, Circular...",270.0,['Modern Australian'],-33.856784,https://www.zomato.com/sydney/bennelong-restau...,151.215297,02 9240 8000,4.9,Excellent,Circular Quay,Bennelong Restaurant,"['Fine Dining', 'Bar']",278.0,False,#7f2704,22.235772,#4186f4
4,"20 Campbell Street, Chinatown, Sydney",55.0,"['Thai', 'Salad']",-33.879035,https://www.zomato.com/sydney/chat-thai-chinatown,151.206409,02 8317 4811,4.5,Excellent,Chinatown,Chat Thai,['Casual Dining'],2150.0,False,#a83703,5.630081,#6f706b


In [12]:
df.dtypes

address           object
cost             float64
cuisine           object
lat              float64
link              object
lng              float64
phone             object
rating_number    float64
rating_text       object
subzone           object
title             object
type              object
votes            float64
groupon             bool
color             object
cost_2           float64
cuisine_color     object
dtype: object

In [39]:
df["cuisine"].dtype
#dtype("0") means object

dtype('O')

In [43]:
# check = df["cuisine"].unique()
# for item in check:
#     print(item)
#not helpful

In [22]:
type(df["cuisine"][0])

str

In [33]:
temp = df["cuisine"][0]
temp
# string

"['Hot Pot', 'Korean BBQ', 'BBQ', 'Korean']"

In [36]:
import re
#pattern = r'[\'\[\],]'
pattern = r',\s*' #removing comma(,) followed by immediate space
cleaned_text = re.sub(pattern, '-', temp)
pattern2 = r'[\'\[\],]'
cleaned_text = re.sub(pattern2, "", cleaned_text)
cleaned_text

'Hot Pot-Korean BBQ-BBQ-Korean'

In [56]:
temp = cleaned_text.split("-")
for a in temp:
    print(a)

Hot Pot
Korean BBQ
BBQ
Korean


In [62]:
# store all cusines in a set
cuisines = set()

#custom function to check all the values
def get_cuisine(str):
    pattern = r',\s*' #removing comma(,) followed by immediate space
    cleaned_text = re.sub(pattern, '-', str)
    pattern2 = r'[\'\[\],]'
    cleaned_text = re.sub(pattern2, "", cleaned_text)
    temp = cleaned_text.split("-")
    for cuisine in temp:
        cuisines.add(cuisine)

# Sydney restaurants sell the following number of cuisines

In [61]:
for value in df["cuisine"]:
    get_cuisine(value)
len(cuisines)

135